# Fast2SMS Bulk Message Sender

This notebook helps you send bulk SMS messages using Fast2SMS service with multiple templates.

---

## Setup Instructions

Before running this notebook, install required packages:

```bash
pip install pandas openpyxl requests ipywidgets
```

---

## 📋 STEP 1: Template Configuration

### How to Add a New Template (For Non-Technical Users):

1. **Copy** an existing template entry below
2. **Paste** it inside the `TEMPLATES` dictionary
3. **Update** these 4 values:
   - `Template ID` (first number) - Get from Fast2SMS dashboard
   - `"sender"` - Your approved sender ID
   - `"vars"` - Count how many {#var#} placeholders in your text
   - `"text"` - Your SMS template text (copy exactly from Fast2SMS)
   - `"dlt_id"` - Your DLT template ID

### Example:
```python
123456: {
    "sender": "MSWAST",
    "vars": 3,
    "text": "Dear {#var#}, your appointment is on {#var#} at {#var#}.",
    "dlt_id": 1234567890123456789
},
```

⚠️ **Important**: 
- Don't forget the comma (`,`) at the end of each template entry
- Keep the format exactly as shown
- Count variables carefully!

In [1]:
import os
import time
import json
import requests
import pandas as pd
from pathlib import Path
from datetime import datetime
from IPython.display import display, HTML, clear_output
import warnings
warnings.filterwarnings('ignore')

# Template definitions - ADD YOUR TEMPLATES HERE
TEMPLATES = {
    173865: {
        "sender": "MSWAST",
        "vars": 6,
        "text": "Dear {#var#} customer {#var#}, you are covered with policy {#var#}, https://c0i.in/MSWAST/{#var#}. Find your nearest clinic https://mswast-map.glitch.me/map2.html  -MSWASTH -MSWASTH",
        "dlt_id": 1107172768973504659
    },
    173903: {
        "sender": "MSWAST",
        "vars": 6,
        "text": "प्रिय {#var#} ग्राहक {#var#}, आप {#var#} बीमा के अंतर्गत आते हैं, https://c0i.in/MSWAST/{#var#}. अधिक जानकारी के लिए कृपया {#var#} पर कॉल करें -MSWASTH",
        "dlt_id": 1107172768969047732
    },
    173904: {
        "sender": "MSWAST",
        "vars": 6,
        "text": "ಆತ್ಮೀಯ {#var#} ಗ್ರಾಹಕ {#var#}, ನೀವು {#var#} ವಿಮೆಯಡಿ ಒಳಗಾಗಿದ್ದೀರಿ, https://c0i.in/MSWAST/{#var#}. ಯಾವುದೇ ವಿವರಗಳಿಗಾಗಿ ದಯವಿಟ್ಟು {#var#} ಗೆ ಕರೆ ಮಾಡಿ -MSWASTH -MSWASTH",
        "dlt_id": 1107172768964776740
    },
    174779: {
        "sender": "MSWAST",
        "vars": 6,
        "text": "அன்புள்ள {#var#} வாடிக்கையாளர்களே {#var#}, உங்கள் {#var#} கொள்கையின் கீழ் பாதுகாக்கப்படுகிறீர்கள், https://c0i.in/MSWAST/{#var#}. பற்றிய விவரங்களுக்கு {#var#} ஜ அழைக்கவும் -MSWASTH",
        "dlt_id": 1107172768960388249
    },
    174780: {
        "sender": "MSWAST",
        "vars": 6,
        "text": "প্রিয় {#var#} গ্রাহক {#var#}, আপনি  {#var#} স্কিমের অধীনে আবৃত, https://c0i.in/MSWAST/{#var#}. যেকোন বিবরণের জন্য অনুগ্রহ করে কল করুন {#var#} -MSWASTH",
        "dlt_id": 1107172768955025536
    },
    174781: {
        "sender": "MSWAST",
        "vars": 6,
        "text": "പ്രിയപ്പെട്ട {#var#} ഉപഭോക്താവേ {#var#}, നിങ്ങൾ {#var#} പോളിസിയിൽ ഉൾപ്പെട്ടിരിക്കുന്നു, https://c0i.in/MSWAST/{#var#}. ഏതെങ്കിലും വിശദാംശങ്ങൾക്ക് ദയവായി വിളിക്കുക {#var#} -MSWASTH",
        "dlt_id": 1107172768941070312
    },
    175561: {
        "sender": "MSWAST",
        "vars": 6,
        "text": "ప్రియమైన {#var#} కస్టమర్ {#var#}, మీరు {#var#} పాలసీ కింద కవరై ఉన్నారు, https://c0i.in/MSWAST/{#var#} సందర్శించండి లేదా {#var#} కి కాల్ చేయండి -MSWASTH",
        "dlt_id": 1107173165717883763
    },
    175562: {
        "sender": "MSWAST",
        "vars": 6,
        "text": "प्रिय {#var#} ग्राहक {#var#}, तुम्ही {#var#} विमा अंतर्गत आहात, https://c0i.in/MSWAST/{#var#}. कोणत्याही तपशिलांसाठी कृपया {#var#} वर कॉल करा -MSWASTH",
        "dlt_id": 1107173165711431532
    },
    181629: {
        "sender": "MSWAST",
        "vars": 5,
        "text": "Dear {#var#} customer {#var#}, you are covered with policy {#var#}, https://c0i.in/MSWAST/{#var#}. Find your nearest clinic {#var#} -MSWASTH",
        "dlt_id": 1107174185289525526
    },
    183078: {
        "sender": "MSWAST",
        "vars": 4,
        "text": "Dear {#var#} customer {#var#}, you are covered with policy {#var#}, https://c0i.in/MSWAST/{#var#}. Find your nearest clinic https://mswast-map.glitch.me/map2.html  -MSWASTH -MSWASTH",
        "dlt_id": 1107174410935731322
    },
    # ============================================
    # NEW TELE HEALTH TEMPLATES (5 variables)
    # ============================================
    201352: {
        "sender": "MSWAST",
        "vars": 5,
        "text": "ಆತ್ಮೀಯ {#var#} ಗ್ರಾಹಕರೇ {#var#}, ನೀವು {#var#}ನಲ್ಲಿ M-Swasth ನಿಂದ ಟೆಲಿ ಹೆಲ್ತ್ ಸೇವೆಗಳಿಂದ ಒಳಗೊಳ್ಳಲ್ಪಟ್ಟಿದ್ದೀರಿ. ಚಂದಾದಾರಿಕೆ ಐಡಿ {#var#}. ನಮ್ಮ ವೈದ್ಯರೊಂದಿಗೆ ಸಂಪರ್ಕ ಸಾಧಿಸಲು ನೀವು {#var#} ಗೆ ಕರೆ ಮಾಡಬಹುದು, 24*7 ಅಥವಾ ಲಿಂಕ್‌ನೊಂದಿಗೆ ಮೊಬೈಲ್ ಅಪ್ಲಿಕೇಶನ್ ಡೌನ್‌ಲೋಡ್ ಮಾಡಿ - https://play.google.com/store/apps/details?id=in.m_insure.patient_app -MSWASTH",
        "dlt_id": 1107172768973504659
    },
    201353: {
        "sender": "MSWAST",
        "vars": 5,
        "text": "ప్రియమైన {#var#} వినియోగధారులు {#var#}, మీరు {#var#}లో M-Swasth ద్వారా టెలి హెల్త్ సర్వీసెస్‌తో కవర్ చేయబడ్డారు. సబ్‌స్క్రిప్షన్ ఐడి {#var#}. మీరు మా వైద్యులతో కనెక్ట్ అవ్వడానికి {#var#} కు కాల్ చేయవచ్చు, 24*7 లేదా లింక్‌తో మొబైల్ యాప్‌ను డౌన్‌లోడ్ చేసుకోవచ్చు - https://play.google.com/store/apps/details?id=in.m_insure.patient_app -MSWASTH",
        "dlt_id": 1107172768973504659
    },
    201354: {
        "sender": "MSWAST",
        "vars": 5,
        "text": "அன்புள்ள {#var#} வாடிக்கையாளர் {#var#}, {#var#}இல் M-Swasth வழங்கும் டெலி ஹெல்த் சர்வீசஸ் மூலம் நீங்கள் பாதுகாக்கப்படுகிறீர்கள். சந்தா ஐடி {#var#}. எங்கள் மருத்துவர்களுடன் தொடர்பு கொள்ள {#var#} 24*7 என்ற எண்ணை நீங்கள் அழைக்கலாம் அல்லது https://play.google.com/store/apps/details?id=in.m_insure.patient_app என்ற இணைப்பில் மொபைல் செயலியைப் பதிவிறக்கலாம் -MSWASTH",
        "dlt_id": 1107172768973504659
    },
    201355: {
        "sender": "MSWAST",
        "vars": 5,
        "text": "പ്രിയപ്പെട്ട {#var#} ഉപഭോക്താവേ {#var#}, {#var#}ൽ M-Swasth-ന്റെ ടെലി ഹെൽത്ത് സർവീസസ് നിങ്ങൾക്ക് ലഭ്യമാണ്. സബ്‌സ്‌ക്രിപ്‌ഷൻ ഐഡി {#var#}. ഞങ്ങളുടെ ഡോക്ടർമാരുമായി ബന്ധപ്പെടാൻ നിങ്ങൾക്ക് {#var#} 24*7 എന്ന നമ്പറിൽ വിളിക്കാം അല്ലെങ്കിൽ https://play.google.com/store/apps/details?id=in.m_insure.patient_app എന്ന ലിങ്ക് ഉപയോഗിച്ച് മൊബൈൽ ആപ്പ് ഡൗൺലോഡ് ചെയ്യാം -MSWASTH",
        "dlt_id": 1107172768973504659
    },
    201356: {
        "sender": "MSWAST",
        "vars": 5,
        "text": "प्रिय {#var#} ग्राहक {#var#}, आप {#var#} में M-Swasth द्वारा टेली हेल्थ सेवाओं के अंतर्गत संरक्षित हैं। आपकी सदस्यता आईडी {#var#} है। आप हमारे डॉक्टरों से 24*7 जुड़ने के लिए {#var#} पर कॉल कर सकते हैं या इस लिंक से मोबाइल ऐप डाउनलोड कर सकते हैं - https://play.google.com/store/apps/details?id=in.m_insure.patient_app -MSWASTH",
        "dlt_id": 1107172768973504659
    },
    201357: {
        "sender": "MSWAST",
        "vars": 5,
        "text": "Dear {#var#} client {#var#}, you are covered with Tele Health Services by M-Swasth in {#var#}. Subscription Id {#var#}. You can call on {#var#} to get connected with our doctors, 24*7 or download mobile app with the link - https://play.google.com/store/apps/details?id=in.m_insure.patient_app -MSWASTH",
        "dlt_id": 1107172768973504659
    }
}

BASE_URL = "https://www.fast2sms.com/dev/bulkV2"

print("✅ Libraries imported successfully!")
print(f"✅ {len(TEMPLATES)} templates loaded")

✅ Libraries imported successfully!
✅ 16 templates loaded


## 📊 STEP 2: View Templates & Generate Sample File

This will show you all available templates in a clear format and generate a sample CSV file for you to fill.

In [2]:
# Create a nicely formatted DataFrame of templates
tpl_data = []
for tpl_id, info in TEMPLATES.items():
    tpl_data.append({
        "Template ID": tpl_id,
        "Sender": info["sender"],
        "Variables": info["vars"],
        "DLT ID": info["dlt_id"],
        "Message Preview": info["text"][:80] + "..." if len(info["text"]) > 80 else info["text"]
    })

tpl_df = pd.DataFrame(tpl_data)

print("="*100)
print("📋 APPROVED SMS TEMPLATES")
print("="*100)
print()

# Display with better formatting
display(HTML(tpl_df.to_html(index=False, max_cols=None)))

print("\n" + "="*100)
print(f"Total Templates Available: {len(TEMPLATES)}")
print("="*100)

# Generate sample file
current_dir = os.getcwd()
sample_filename = "sample_sms_data.csv"
sample_path = os.path.join(current_dir, sample_filename)

# Create columns for up to 10 variables
columns = ["mobile"] + [f"v{i}" for i in range(1, 11)] + ["template_id"]

# Create sample with example row
sample_data = pd.DataFrame([
    ["9876543210", "John", "12345", "Health Plus", "ABC123", "1800-XXX-XXX", "", "", "", "", "", "173865"],
    ["9876543211", "Jane", "67890", "Care Pro", "XYZ456", "1800-YYY-YYY", "", "", "", "", "", "173865"]
], columns=columns)

sample_data.to_csv(sample_path, index=False)

print("\n✅ SAMPLE FILE GENERATED")
print(f"📁 Location: {sample_path}")
print(f"📄 Filename: {sample_filename}")
print("\n📝 Instructions:")
print("   1. Open the sample CSV file in Excel or any spreadsheet software")
print("   2. Fill in your data:")
print("      - mobile: 10-digit mobile number (without +91)")
print("      - v1 to v10: Variable values for your template")
print("      - template_id: Choose from the Template IDs shown above")
print("   3. Save the file and continue to the next step")
print("\n💡 Tip: Only fill variables (v1, v2, etc.) that your template uses.")
print("    For example, if template needs 4 variables, only fill v1 to v4.")

📋 APPROVED SMS TEMPLATES



Template ID,Sender,Variables,DLT ID,Message Preview
173865,MSWAST,6,1107172768973504659,"Dear {#var#} customer {#var#}, you are covered with policy {#var#}, https://c0i...."
173903,MSWAST,6,1107172768969047732,"प्रिय {#var#} ग्राहक {#var#}, आप {#var#} बीमा के अंतर्गत आते हैं, https://c0i.in..."
173904,MSWAST,6,1107172768964776740,"ಆತ್ಮೀಯ {#var#} ಗ್ರಾಹಕ {#var#}, ನೀವು {#var#} ವಿಮೆಯಡಿ ಒಳಗಾಗಿದ್ದೀರಿ, https://c0i.in..."
174779,MSWAST,6,1107172768960388249,"அன்புள்ள {#var#} வாடிக்கையாளர்களே {#var#}, உங்கள் {#var#} கொள்கையின் கீழ் பாதுகா..."
174780,MSWAST,6,1107172768955025536,"প্রিয় {#var#} গ্রাহক {#var#}, আপনি {#var#} স্কিমের অধীনে আবৃত, https://c0i.in/..."
174781,MSWAST,6,1107172768941070312,"പ്രിയപ്പെട്ട {#var#} ഉപഭോക്താവേ {#var#}, നിങ്ങൾ {#var#} പോളിസിയിൽ ഉൾപ്പെട്ടിരിക്..."
175561,MSWAST,6,1107173165717883763,"ప్రియమైన {#var#} కస్టమర్ {#var#}, మీరు {#var#} పాలసీ కింద కవరై ఉన్నారు, https://..."
175562,MSWAST,6,1107173165711431532,"प्रिय {#var#} ग्राहक {#var#}, तुम्ही {#var#} विमा अंतर्गत आहात, https://c0i.in/M..."
181629,MSWAST,5,1107174185289525526,"Dear {#var#} customer {#var#}, you are covered with policy {#var#}, https://c0i...."
183078,MSWAST,4,1107174410935731322,"Dear {#var#} customer {#var#}, you are covered with policy {#var#}, https://c0i...."



Total Templates Available: 16

✅ SAMPLE FILE GENERATED
📁 Location: d:\M-SWASTH\EDWIN\fast2smsv2\sample_sms_data.csv
📄 Filename: sample_sms_data.csv

📝 Instructions:
   1. Open the sample CSV file in Excel or any spreadsheet software
   2. Fill in your data:
      - mobile: 10-digit mobile number (without +91)
      - v1 to v10: Variable values for your template
      - template_id: Choose from the Template IDs shown above
   3. Save the file and continue to the next step

💡 Tip: Only fill variables (v1, v2, etc.) that your template uses.
    For example, if template needs 4 variables, only fill v1 to v4.


## 📂 STEP 3: Load Your Data File

Choose how you want to provide your data file:

In [3]:
print("="*100)
print("📂 DATA FILE LOADING OPTIONS")
print("="*100)
print("\nOption 1: Upload file (drag & drop or browse)")
print("Option 2: Paste file path directly")
print("\nWhich option do you prefer?")
print("\n[1] File Upload (GUI)")
print("[2] Enter Path (Text)")
print("\nEnter your choice (1 or 2):")

📂 DATA FILE LOADING OPTIONS

Option 1: Upload file (drag & drop or browse)
Option 2: Paste file path directly

Which option do you prefer?

[1] File Upload (GUI)
[2] Enter Path (Text)

Enter your choice (1 or 2):


In [5]:
# Get user choice
choice = input("Your choice (1 or 2): ").strip()

df = None
file_path = None

if choice == "1":
    # Option 1: File Upload Widget
    print("\n🔄 Loading file upload widget...")
    try:
        from ipywidgets import FileUpload
        from IPython.display import display
        import io
        
        uploader = FileUpload(
            accept='.csv,.xlsx',
            multiple=False,
            description='Choose File'
        )
        
        print("\n📤 Click the button below to upload your CSV or XLSX file:")
        display(uploader)
        
        print("\n⏳ Waiting for file upload... (Click 'Upload' button after selecting)")
        print("\nAfter uploading, run the NEXT cell to process the file.")
        
    except ImportError:
        print("\n❌ File upload widget not available. Please use Option 2 instead.")
        print("\nTo enable file upload, run: pip install ipywidgets")
        print("Then restart the kernel and try again.")

elif choice == "2":
    # Option 2: Paste Path
    print("\n📝 Enter the full path to your CSV or XLSX file:")
    print("\nExample:")
    print("  - Windows: C:\\Users\\YourName\\Documents\\data.csv")
    print("  - Mac/Linux: /Users/YourName/Documents/data.csv")
    print("  - Current folder: just enter filename.csv")
    print()
    
    file_path = input("File path: ").strip().strip('"').strip("'")
    
    # Try to load the file
    try:
        ext = Path(file_path).suffix.lower()
        
        print(f"\n🔄 Loading file: {file_path}")
        
        if ext == ".xlsx":
            df = pd.read_excel(file_path, dtype=str).fillna("")
        else:
            df = pd.read_csv(file_path, dtype=str).fillna("")
        
        # Normalize column names
        df.columns = [c.strip().lower() for c in df.columns]
        
        # Ensure all v1-v10 columns exist
        for i in range(1, 11):
            col = f"v{i}"
            if col not in df.columns:
                df[col] = ""
        
        # Reorder columns
        df = df[["mobile"] + [f"v{i}" for i in range(1, 11)] + ["template_id"]]
        
        print("\n✅ FILE LOADED SUCCESSFULLY!")
        print("="*100)
        print(f"📊 Total Rows: {len(df)}")
        print(f"📱 Unique Mobile Numbers: {df['mobile'].nunique()}")
        print("\n📋 First 5 rows preview:")
        display(HTML(df.head().to_html(index=False)))
        
        # Validate template IDs
        invalid_templates = []
        for idx, row in df.iterrows():
            try:
                tid = int(float(str(row['template_id']).strip()))
                if tid not in TEMPLATES:
                    invalid_templates.append((idx+1, tid))
            except:
                invalid_templates.append((idx+1, row['template_id']))
        
        if invalid_templates:
            print(f"\n⚠️ WARNING: {len(invalid_templates)} rows have invalid template IDs:")
            for row_num, tid in invalid_templates[:5]:
                print(f"   Row {row_num}: Template ID '{tid}' not found")
            if len(invalid_templates) > 5:
                print(f"   ... and {len(invalid_templates) - 5} more")
        else:
            print("\n✅ All template IDs are valid!")
        
    except FileNotFoundError:
        print(f"\n❌ ERROR: File not found at: {file_path}")
        print("\nPlease check:")
        print("  1. The file path is correct")
        print("  2. The file exists at that location")
        print("  3. You have permission to access the file")
    except Exception as e:
        print(f"\n❌ ERROR loading file: {str(e)}")
        print("\nPlease ensure:")
        print("  1. The file is a valid CSV or XLSX file")
        print("  2. The file is not open in another program")
        print("  3. Required columns exist: mobile, template_id")

else:
    print("\n❌ Invalid choice. Please enter 1 or 2.")


📝 Enter the full path to your CSV or XLSX file:

Example:
  - Windows: C:\Users\YourName\Documents\data.csv
  - Mac/Linux: /Users/YourName/Documents/data.csv
  - Current folder: just enter filename.csv


🔄 Loading file: D:\M-SWASTH\EDWIN\fast2smsv2\sample_sms_data.csv

✅ FILE LOADED SUCCESSFULLY!
📊 Total Rows: 3
📱 Unique Mobile Numbers: 3

📋 First 5 rows preview:


mobile,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,template_id
8700612665,satvick,satvick,satvick,satvick,satvick,,,,,,201357
9760698397,raj,raj,raj,raj,raj,,,,,,201357
8072089185,edwin,edwin,edwin,edwin,edwin,,,,,,201356



✅ All template IDs are valid!


In [6]:
# If Option 1 (file upload) was chosen, process the uploaded file here
if choice == "1" and 'uploader' in locals():
    if uploader.value:
        import io
        
        uploaded_file = list(uploader.value.values())[0]
        filename = list(uploader.value.keys())[0]
        content = uploaded_file['content']
        
        print(f"\n🔄 Processing uploaded file: {filename}")
        
        try:
            ext = Path(filename).suffix.lower()
            
            if ext == ".xlsx":
                df = pd.read_excel(io.BytesIO(content), dtype=str).fillna("")
            else:
                df = pd.read_csv(io.BytesIO(content), dtype=str).fillna("")
            
            # Normalize column names
            df.columns = [c.strip().lower() for c in df.columns]
            
            # Ensure all v1-v10 columns exist
            for i in range(1, 11):
                col = f"v{i}"
                if col not in df.columns:
                    df[col] = ""
            
            # Reorder columns
            df = df[["mobile"] + [f"v{i}" for i in range(1, 11)] + ["template_id"]]
            
            print("\n✅ FILE LOADED SUCCESSFULLY!")
            print("="*100)
            print(f"📊 Total Rows: {len(df)}")
            print(f"📱 Unique Mobile Numbers: {df['mobile'].nunique()}")
            print("\n📋 First 5 rows preview:")
            display(HTML(df.head().to_html(index=False)))
            
            # Validate template IDs
            invalid_templates = []
            for idx, row in df.iterrows():
                try:
                    tid = int(float(str(row['template_id']).strip()))
                    if tid not in TEMPLATES:
                        invalid_templates.append((idx+1, tid))
                except:
                    invalid_templates.append((idx+1, row['template_id']))
            
            if invalid_templates:
                print(f"\n⚠️ WARNING: {len(invalid_templates)} rows have invalid template IDs:")
                for row_num, tid in invalid_templates[:5]:
                    print(f"   Row {row_num}: Template ID '{tid}' not found")
                if len(invalid_templates) > 5:
                    print(f"   ... and {len(invalid_templates) - 5} more")
            else:
                print("\n✅ All template IDs are valid!")
                
        except Exception as e:
            print(f"\n❌ ERROR processing file: {str(e)}")
            df = None
    else:
        print("\n⚠️ No file uploaded yet. Please upload a file in the previous cell first.")
        df = None
elif choice == "2":
    print("\n✅ File already processed. You can proceed to the next step.")
else:
    print("\n⚠️ Please complete Step 3 first by choosing option 1 or 2.")


✅ File already processed. You can proceed to the next step.


## 🚀 STEP 4: Send SMS Messages

Configure your API settings and send the messages.

In [7]:
# Check if data is loaded
if df is None or df.empty:
    print("❌ ERROR: No data loaded. Please complete Step 3 first.")
else:
    print("="*100)
    print("🔐 API CONFIGURATION")
    print("="*100)
    print()
    
    # Get API key
    api_key = os.getenv("F2S_KEY")
    if api_key:
        print("✅ API Key found in environment variable (F2S_KEY)")
        print(f"   Key: {api_key[:10]}...{api_key[-4:]}")
        use_env_key = input("\nUse this key? (y/n): ").strip().lower()
        if use_env_key != 'y':
            api_key = input("\nEnter your Fast2SMS API key: ").strip()
    else:
        print("ℹ️ API Key not found in environment")
        api_key = input("\nEnter your Fast2SMS API key: ").strip()
    
    # Schedule time (optional)
    print("\n" + "-"*100)
    print("⏰ SCHEDULE OPTIONS")
    print("-"*100)
    print("\nDo you want to schedule the messages for later?")
    print("Format: YYYY-MM-DD-HH-MM (e.g., 2024-12-25-14-30 for Dec 25, 2024 at 2:30 PM)")
    print("Leave blank to send immediately.")
    schedule_time = input("\nSchedule time (or press Enter to skip): ").strip()
    
    if schedule_time:
        print(f"\n✅ Messages will be scheduled for: {schedule_time}")
    else:
        print("\n✅ Messages will be sent immediately")
    
    flash = "0"  # Normal SMS (not flash)
    
    print("\n" + "="*100)
    print("📊 SENDING SUMMARY")
    print("="*100)
    print(f"Total messages to send: {len(df)}")
    print(f"Unique numbers: {df['mobile'].nunique()}")
    print("\nReady to send? This action cannot be undone.")
    
    confirm = input("\nType 'YES' to proceed: ").strip()
    
    if confirm.upper() == "YES":
        print("\n🚀 Starting SMS sending process...\n")
    else:
        print("\n❌ Operation cancelled.")

🔐 API CONFIGURATION

ℹ️ API Key not found in environment

----------------------------------------------------------------------------------------------------
⏰ SCHEDULE OPTIONS
----------------------------------------------------------------------------------------------------

Do you want to schedule the messages for later?
Format: YYYY-MM-DD-HH-MM (e.g., 2024-12-25-14-30 for Dec 25, 2024 at 2:30 PM)
Leave blank to send immediately.

✅ Messages will be sent immediately

📊 SENDING SUMMARY
Total messages to send: 3
Unique numbers: 3

Ready to send? This action cannot be undone.

🚀 Starting SMS sending process...



In [8]:
# Helper functions
def need_unicode(txt):
    """Check if text contains non-ASCII characters"""
    return any(ord(c) > 127 for c in txt)

def send_with_retry(payload, api_key, tries=3, base_pause=1.0):
    """Send SMS with retry logic"""
    for n in range(tries):
        try:
            r = requests.post(
                BASE_URL, 
                json=payload,
                headers={"authorization": api_key},
                timeout=12
            )
            js = r.json()
            if js.get("return") is True:
                return True, js.get("request_id"), "Success", js
            else:
                err = js.get("message", str(js))
                return False, None, err, js
        except requests.exceptions.Timeout:
            err = "Request timeout"
        except requests.exceptions.RequestException as e:
            err = f"Network error: {str(e)}"
        except Exception as e:
            err = f"Error: {str(e)}"
        
        if n < tries - 1:
            time.sleep(base_pause * (2**n))
    
    return False, None, err, {"error": err}

print("✅ Helper functions loaded")

✅ Helper functions loaded


In [9]:
# Send SMS messages
if confirm.upper() == "YES" and df is not None:
    results = []
    start_time = datetime.now()
    
    print("="*100)
    print("📤 SENDING SMS MESSAGES")
    print("="*100)
    print(f"Started at: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print()
    print(f"{'Row':<6} {'Mobile':<15} {'Template':<10} {'Status':<10} {'Request ID':<20} {'Message'}")
    print("-"*100)
    
    success_count = 0
    fail_count = 0
    
    for idx, row in df.iterrows():
        row_num = idx + 1
        mobile = str(row["mobile"]).strip()
        
        # Validate and get template
        try:
            msg_id = int(float(str(row["template_id"]).strip()))
        except:
            status = "FAIL"
            message = f"Invalid template_id: {row['template_id']}"
            results.append({
                "row": row_num,
                "mobile": mobile,
                "template_id": row['template_id'],
                "status": status,
                "request_id": "",
                "message": message,
                "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            })
            print(f"{row_num:<6} {mobile:<15} {str(row['template_id']):<10} {status:<10} {'N/A':<20} {message}")
            fail_count += 1
            continue
        
        tpl = TEMPLATES.get(msg_id)
        if not tpl:
            status = "FAIL"
            message = f"Template {msg_id} not found"
            results.append({
                "row": row_num,
                "mobile": mobile,
                "template_id": msg_id,
                "status": status,
                "request_id": "",
                "message": message,
                "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            })
            print(f"{row_num:<6} {mobile:<15} {msg_id:<10} {status:<10} {'N/A':<20} {message}")
            fail_count += 1
            continue
        
        # Build variables
        nvars = tpl["vars"]
        vals = []
        for i in range(1, nvars + 1):
            val = str(row[f"v{i}"]).strip()
            if val.lower() in {"n/a", "na", "none", "null"}:
                val = ""
            vals.append(val)
        vars_pipe = "|".join(vals)
        
        # Build payload
        payload = {
            "route": "dlt",
            "sender_id": tpl["sender"],
            "message": str(msg_id),
            "variables_values": vars_pipe,
            "numbers": mobile,
            "flash": flash
        }
        
        if schedule_time:
            payload["schedule_time"] = schedule_time
        
        if need_unicode(tpl["text"] + vars_pipe):
            payload["language"] = "unicode"
        
        # Send SMS
        ok, req_id, message, raw = send_with_retry(payload, api_key)
        
        status = "SUCCESS" if ok else "FAIL"
        if ok:
            success_count += 1
        else:
            fail_count += 1
        
        results.append({
            "row": row_num,
            "mobile": mobile,
            "template_id": msg_id,
            "status": status,
            "request_id": req_id or "",
            "message": message,
            "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "raw_response": str(raw)
        })
        
        # Display progress
        req_id_display = (req_id[:17] + "...") if req_id and len(req_id) > 20 else (req_id or "N/A")
        msg_display = message[:40] if len(message) > 40 else message
        print(f"{row_num:<6} {mobile:<15} {msg_id:<10} {status:<10} {req_id_display:<20} {msg_display}")
        
        # Small delay to avoid rate limiting
        time.sleep(0.2)
    
    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()
    
    # Save results to Excel in current directory
    current_dir = os.getcwd()
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_filename = f"sms_log_{timestamp}.xlsx"
    log_path = os.path.join(current_dir, log_filename)
    
    log_df = pd.DataFrame(results)
    log_df.to_excel(log_path, index=False, engine='openpyxl')
    
    # Print summary report
    print("\n" + "="*100)
    print("📊 FINAL REPORT")
    print("="*100)
    print(f"\n⏱️  Time taken: {duration:.2f} seconds")
    print(f"📤 Total messages attempted: {len(df)}")
    print(f"✅ Successful: {success_count} ({success_count/len(df)*100:.1f}%)")
    print(f"❌ Failed: {fail_count} ({fail_count/len(df)*100:.1f}%)")
    print(f"\n📁 Log file saved to:")
    print(f"   {log_path}")
    print(f"\n📄 Filename: {log_filename}")
    
    if fail_count > 0:
        print(f"\n⚠️  {fail_count} messages failed. Check the log file for details.")
        print("\nCommon failure reasons:")
        print("   • Invalid mobile numbers")
        print("   • Insufficient API balance")
        print("   • Invalid template ID")
        print("   • DLT/template mismatch")
    
    print("\n" + "="*100)
    print("✅ Process completed successfully!")
    print("="*100)
    
else:
    print("\n⚠️ Sending process not initiated. Please complete the previous steps.")

📤 SENDING SMS MESSAGES
Started at: 2025-11-04 14:04:51

Row    Mobile          Template   Status     Request ID           Message
----------------------------------------------------------------------------------------------------
1      8700612665      201357     SUCCESS    CuvDF0V6ig9WaEf      Success
2      9760698397      201357     SUCCESS    9jefz60pFDPgv2V      Success
3      8072089185      201356     SUCCESS    DhXIfJ0FLjop6yS      Success

📊 FINAL REPORT

⏱️  Time taken: 4.57 seconds
📤 Total messages attempted: 3
✅ Successful: 3 (100.0%)
❌ Failed: 0 (0.0%)

📁 Log file saved to:
   d:\M-SWASTH\EDWIN\fast2smsv2\sms_log_20251104_140456.xlsx

📄 Filename: sms_log_20251104_140456.xlsx

✅ Process completed successfully!


## 🎉 Done!

Your SMS messages have been sent and the log file has been saved to your current directory.

### Next Steps:
1. Check the Excel log file for detailed results
2. Review any failed messages and correct the data if needed
3. Re-run from Step 3 if you need to send to failed numbers again

### Need Help?
- Check Fast2SMS dashboard for delivery status
- Verify your API balance
- Ensure template IDs and DLT IDs match

---

**Created for M-SWASTH Team** | Fast2SMS Bulk Sender v2.0